<h3>Auteur : Nanfa Rochinel-Bienvenu</h3>
<p>le 29/12/2020</p>

#### crunching de data

<p> 
Ce notebook qui sert aussi de rapport est celui dans le quel vous trouverez tous mes scripts repondant respectivement aux question posé. 
</p>
<p>
Dans un premier je vais essayer de charge et filtrer les différentes tables.puis afficher un aperçu de leurs contenus. Ensuite faire les différents traitement necessaire pour répondre de façon éfficace aux questions
</p>

<p><u>ps</u>: Dans chaque cellule du notebook seront détaillé les méthodes utilisé  , et explication des choix que nous avons fait.
</p> 

In [158]:
import pandas as pd
import numpy as np
import numbers

#### Chargement de données

<p> Nous allons dans cette partie charger les differents données , en utilisant la fonction pd.read() de pandas</p>

In [191]:
#lecture des fichiers csv

Cities = pd.read_csv("data/cities.csv")
Providers= pd.read_csv("data/providers.csv")
Ticket = pd.read_csv("data/ticket_data.csv")

In [192]:
# aperçu du contenu de Cities 
Cities 
Cities.head()

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [193]:
#traitement optionnel

citie_id_name = Cities.apply(lambda x : (x['id'] , x['unique_name']) , axis=1)

<p> la variable <b> citie_id_name  </b> contient le couple nom et id je vais utiliser pour la comprehension des dataframe </p>

In [318]:
citie_id_name.head()

0      (5159, padua)
1    (76, barcelona)
2        (81, basel)
3    (259, erlangen)
4      (11979, balș)
dtype: object

In [196]:
#aperçu du contenu de Providers

Providers=Providers.dropna()
Providers.head()

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
8,17,9,31,expressbusMunchen,Expressbus Munchen,False,False,False,False,bus
9,18,9,33,berlinLinienBus,Berlin Linien Bus,False,False,False,False,bus
10,19,9,37,ostfrieslandexpress,OstfrieslandExpress,False,False,False,False,bus
11,20,9,41,icBus,IC Bus,False,False,False,False,bus
12,21,9,45,busandfly,Busandfly,False,False,False,False,bus


In [216]:
#aperçu du contenu Tickets


Ticket=Ticket.dropna() #suppression des valeurs manquantes

#convertion des date

Ticket['departure_ts'] = pd.to_datetime(Ticket['departure_ts'])
Ticket['arrival_ts'] = pd.to_datetime(Ticket['arrival_ts'])
Ticket['search_ts'] = pd.to_datetime(Ticket['search_ts'])


Ticket.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542
49,6795074,13,767.0,396.0,2017-10-06 05:25:00+00:00,2017-10-06 16:40:00+00:00,1990,2017-10-01 01:04:12.45+00,"{932,932}",{13},628,453


In [217]:
type (Ticket['departure_ts'][1])

pandas._libs.tslibs.timestamps.Timestamp

### recherche du prix max , moyen et min de chaque trajet

<p> étant donnée que j'ai pas eu la documentation concernant les différents feature de chaque dataframe <br/> je considère que <br/> 
<font COLOR="blue"> o_city </font> : ville de depart <br/> <font COLOR="blue"> d_city </font> : ville d'arrivé </p>

<p>
       Dans un deuxième temps on remarque que plusieurs compagnies font les même trajet , donc ce qui nous amène à comprendre qu'on peut peut regrouper nos donner par traget (<font COLOR = "red"> fonction <b>groupby</b> de pandas </font>) , ce qui va nous facilité le calcule du mean  , max et moyenne de chaque traget . puis je vais utiliser une aggregation de type <font COLOR = "red"> agg </font> pour mettre tout cela dans une dataframe. 
</p>

In [226]:
df_prix = Ticket.groupby(['o_city','d_city'] ).agg({'price_in_cents': ['mean', 'min', 'max']})

In [207]:
# fonction pour matcher le nom et les id

def _id_name_match(x,y):
    x=list(x)
    for i in y:
        if (x[0]==i[0]):
            x[0]=i[1]
    for i in y:
        if (x[1]==i[0]):
            x[1]=i[1]
    return tuple(x)


def draw_trajet(x):
    x=list(x)
    x[0]=x[0]+"-->"
    return ''.join(tuple(x))

In [227]:
# ajout de la colonne trajet à la table df_prix (colone qui contient la moyenne , le min et max du prix de chaque trajet) 

df_prix['trajet']=df_prix.index.to_series().apply(lambda x : _id_name_match(x,citie_id_name)).apply(draw_trajet) 

In [228]:
df_prix.head()

price_in_cents                          trajet
                        mean    min    max                  
o_city d_city                                               
5      23       20320.000000  18600  22000  agde-->amsterdam
6      227      11755.000000   9860  13650      agen-->dijon
       504       4041.538462   2000   8920  agen-->marseille
       628       2797.500000   2600   3190      agen-->paris
       845       2420.000000   2420   2420   agen-->toulouse

<h5> <u> resumé </u>: </h5>

<p>le dataframe <b> df_prix</b> est le df contenant chaque destination ainsi la moyenne , le min , et le max  des prix dans ceux-ci  .<p>
<p> le traget agde pour amsterdam a pour prix moyen :  <font COLOR = 'blue'> 20320 </font> , prix minimal : <font COLOR = 'blue'> 18600 </font>, prix maximal : <font COLOR = 'blue'> 22000 </font> </p> 

### Determination de la durée max , min et moyenne de chaque trajet 

<p> dans cette partie je vais appliquer la même méthode que j'ai faite pour repondre à la question précédante.</br>
à la seul difference je suis obligé de calculer la durée de chaque trajet en faisant le <b>trajet de depart <font COLOR = "GREEN">-</font> le trajet d'arrivé </b> 
</p>
<p> En suite on sera obligé de faire des convertions , enfin d'utiliser les aggrégation pour calculer le max , min et mean des durée des trajets</p>

In [302]:
Ticket_copy =  Ticket.copy() 

In [303]:
Ticket_copy["Duration"] = Ticket['arrival_ts'] - Ticket['departure_ts']

In [304]:
Ticket_copy.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city,Duration
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542,0 days 17:50:00
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542,1 days 07:57:00
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542,0 days 21:35:00
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542,0 days 21:46:00
49,6795074,13,767.0,396.0,2017-10-06 05:25:00+00:00,2017-10-06 16:40:00+00:00,1990,2017-10-01 01:04:12.45+00,"{932,932}",{13},628,453,0 days 11:15:00


In [312]:
# conversion de la durrée en seconde

Ticket_copy['Duration']=Ticket_copy['Duration'].apply(pd.Timedelta.total_seconds)

In [313]:
Ticket_copy['Duration'].apply(lambda x : pd.Timedelta(seconds=x))

1       0 days 17:50:00
2       1 days 07:57:00
3       0 days 21:35:00
4       0 days 21:46:00
49      0 days 11:15:00
              ...      
74163   0 days 13:30:00
74164   0 days 06:30:00
74165   0 days 06:45:00
74166   0 days 07:45:00
74167   0 days 06:55:00
Name: Duration, Length: 32727, dtype: timedelta64[ns]

In [315]:
# calcule du mean , min , max des durées de chaque trajet et reconvertion en Timedelta

df_duration = Ticket_copy.groupby(['o_city','d_city'] ).agg({'Duration': ['mean','min','max']}).apply(lambda x : x.apply(lambda y :pd.Timedelta(seconds=y)),axis=1)

df_duration["trajet"]=df_duration.index.to_series().apply(lambda x : _id_name_match(x,citie_id_name)).apply(draw_trajet) 

In [316]:
df_duration.head()

Duration                                  \
                                mean             min             max   
o_city d_city                                                          
5      23            0 days 10:18:48 0 days 08:53:00 0 days 15:54:00   
6      227           0 days 13:42:30 0 days 12:24:00 0 days 15:01:00   
       504    0 days 08:33:09.230769 0 days 05:36:00 0 days 12:20:00   
       628           0 days 12:10:00 0 days 09:40:00 0 days 14:30:00   
       845           0 days 04:11:00 0 days 04:11:00 0 days 04:11:00   
...                              ...             ...             ...   
11835  173    0 days 07:05:13.333333 0 days 06:31:00 0 days 08:11:00   
11938  126           1 days 03:17:30 0 days 16:45:00 1 days 11:20:00   
12124  1064          0 days 23:10:00 0 days 23:10:00 0 days 23:10:00   
12166  857           0 days 21:55:00 0 days 21:55:00 0 days 21:55:00   
12190  8937          0 days 09:10:00 0 days 09:10:00 0 days 09:10:00   

                                            trajet  
                                                    
o_city d_city                                       
5      23                         agde-->amsterdam  
6      227                            agen-->dijon  
       504                        agen-->marseille  
       628                            agen-->paris  
       845                         agen-->toulouse  
...                                            ...  
11835  173               roissy-en-france-->cannes  
11938  126                      puteaux-->bordeaux  
12124  1064    cherbourg-en-cotentin-->carcassonne  
12166  857          aeroport-paris-vatry--->troyes  
12190  8937                     gerona-->tarragona  

[1221 rows x 4 columns]

<h5> <u> resumé </u>: </h5>

<p>le dataframe <b> df_duration</b> est le df contenant chaque destination ainsi la moyenne , le min , et le max  des durées  de ceux-ci  .<p>
<p> le traget agde pour amsterdam a pour durée  moyenne :  <font COLOR = 'blue'> 10:18:48 </font> , durée  minimale : <font COLOR = 'blue'> 08:53:00 </font>, durée maximale : <font COLOR = 'blue'> 15:54:00 </font> </p> 

###  différence de prix moyen et durée selon le deplacement (Train ou bus)

<p> pour répondre à cette question nous allons dans une premier temps  faire :  </p>
<p>- une fusion des tables entre la df de ticket et celui des providers </p>
<p>- puis faire un groupe by en fonction du type de transport </p>
<p>- pour aller plus loin , on pourra refaire un nouveau traitement sur notre<br/>
nouvelle df resultante pour avoir le prix moyen d'un traget chez une compagnie en focntion du mode de transport que celui si propose et ainsi que la durée moyen aussi</p>
<p><font COLOR = "red">ps: </font> pour repondre complètement à la question les distances seront associé plutard </p>

<h5> <u> rappel </u> : Calcule de la distance entre deux point d'une sphère (<font COLOR = "purple"> haversine</font>) </h5>

<p>
haversine est pour calculer la distance du grand cercle entre deux points, c'est-à-dire <br/>
la distance la plus courte au-dessus de la surface de la terre - donnant une distance «à vol d'oiseau» entre <br/>      les points<br/>
(en ignorant les collines qu'ils survolent, bien sûr!).Haversine<br/>
</p>

<p>
    formule : <b>  a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2) <br/>
    c = 2 ⋅ atan2( √a, √(1−a) )<br/>
    d = R ⋅ c <br/>
    </b>
    <div>
        où <b>φ</b> est la latitude, <b>λ</b> est la longitude, <b>R</b> est le rayon de la Terre <i>(rayon moyen = 6 371 km)</i>;
    notez que les angles doivent être en radians pour passer aux fonctions trigonométriques
        </div>
</p>

In [331]:
# recharger les fichers csv 

df_Cities = pd.read_csv("data/cities.csv")
df_Providers= pd.read_csv("data/providers.csv")
df_Ticket = pd.read_csv("data/ticket_data.csv")

In [332]:
df_Ticket=df_Ticket.rename(columns={"company":"company_id"})

In [333]:
df_Ticket.head()

,id,company_id,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


In [434]:
# fusion de la table providers et des tickets 
df_inter=df_Ticket.merge(df_Providers , how ='inner' , on = "company_id")

In [435]:
df_inter.head()

,id_x,company_id,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,d_city,id_y,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,542,16,NaN,distribusion,Distribusion,False,False,False,False,bus
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,542,17,31,expressbusMunchen,Expressbus Munchen,False,False,False,False,bus
2,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,542,18,33,berlinLinienBus,Berlin Linien Bus,False,False,False,False,bus
3,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,542,19,37,ostfrieslandexpress,OstfrieslandExpress,False,False,False,False,bus
4,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},...,542,20,41,icBus,IC Bus,False,False,False,False,bus


In [336]:
citie_distance = df_inter[["id_x","d_city","o_city"]]

In [339]:
# utilisation de la librairie haversine
#decommentez cette ligne si vous voulez utiliser la bibliothèque  haversine

!pip install haversine 

In [366]:
from haversine import haversine, Unit

In [426]:
# retourne la distance en km d'un trajet en fontionc de id

def distance_trajet(x,df):
    if (df[df['id']==x['o_city']].empty and df[df['id']==x['d_city']].empty):
        return None
    else : 
        a= tuple(df[df['id']==x['o_city']].apply(lambda x: (x['latitude'] , x['longitude']) , axis=1))[0]
        b=tuple (df[df['id']==x['d_city']].apply(lambda x: (x['latitude'] , x['longitude']) , axis=1))[0]
        return haversine(a,b)

In [427]:
# calcule des distance

test = df_inter.apply(lambda x : distance_trajet(x,df_Cities) , axis=1) # a compiler si vous avez du tem si non a ne pas faire ça prend beacoup de temps

In [450]:
# creation de la colone distance dans la table df inter
df_inter["distance"]=test 

In [451]:
# string to date time

df_inter['departure_ts'] = pd.to_datetime(df_inter['departure_ts'])
df_inter['arrival_ts'] = pd.to_datetime(df_inter['arrival_ts'])
df_inter['search_ts'] = pd.to_datetime(df_inter['search_ts'])

In [452]:
# convertion des colones date en seconde  et calcule de la durée

df_inter['duration']=df_inter['arrival_ts']-df_inter['departure_ts'] # calcule de la durée du trajet
df_inter['duration']=df_inter['duration'].apply(pd.Timedelta.total_seconds)

In [453]:
#final table 
df=df_inter.dropna()

In [454]:
df.head()

,id_x,company_id,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,...,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type,distance,duration
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},...,31,expressbusMunchen,Expressbus Munchen,False,False,False,False,bus,503.197862,64200.0
2,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},...,33,berlinLinienBus,Berlin Linien Bus,False,False,False,False,bus,503.197862,64200.0
3,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},...,37,ostfrieslandexpress,OstfrieslandExpress,False,False,False,False,bus,503.197862,64200.0
4,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},...,41,icBus,IC Bus,False,False,False,False,bus,503.197862,64200.0
5,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},...,45,busandfly,Busandfly,False,False,False,False,bus,503.197862,64200.0


<p> le traitement des données étant terminé on peut passer à ce qui est demandé , c'est à dire  la différence moyen des prix et la durée de chaque trajet en fonction du type de déplacement utilisé </p>

<p> ps : j'ai des doutes en ce qui concerne la compréhension de la  dernière question  c'est pour cela que j'ai essayé de la reformuler ainsi</p> 

In [490]:
# difference de prix moyen selon le deplacement bus , train , sur 0-200km

df_200 = df.query('distance>=0' and 'distance>=200').groupby('transport_type').agg({'duration': ['mean'],'price_in_cents' :['mean']})

In [465]:
df_200

,duration,price_in_cents
,mean,mean
transport_type,,
bus,52843.937992,3461.755092
train,45733.333333,3265.555556


In [468]:
df_200['duration']['mean']['bus']

52843.93799156291

In [472]:
mean_duration_200 = pd.Timedelta(seconds = df_200['duration']['mean']['bus']-df_200['duration']['mean']['train'])

In [486]:
print ("la difference de durée moyen entre le bus et le train pour les trajets entre \n 0-200 km est :",mean_duration_200)

la difference de durée moyen entre le bus et le train pour les trajets entre 
 0-200 km est : 0 days 01:58:30.604658


In [481]:
mean_price_2OO =  df_200['price_in_cents']['mean']['bus'] - df_200['price_in_cents']['mean']['train']

In [488]:
print ("la difference de price moyen entre le bus et le train pour les trajets entre \n 0-200 km est :",mean_price_2OO,"cents")

la difference de price moyen entre le bus et le train pour les trajets entre 
 0-200 km est : 196.19953623386118 cents


In [493]:
# difference de prix moyen selon le deplacement bus , train , sur 201-800km

df_800 = df.query('distance>=201' and 'distance<=800').groupby('transport_type').agg({'duration': ['mean'],'price_in_cents' :['mean']})

In [494]:
df_800

,duration,price_in_cents
,mean,mean
transport_type,,
bus,48948.318986,3155.261196
train,45733.333333,3265.555556


In [498]:
mean_price_800 =  df_800['price_in_cents']['mean']['bus'] - df_800['price_in_cents']['mean']['train']

print ("la difference de price moyen entre le bus et le train pour les trajets entre \n 201-800 km est :",(-1)*mean_price_800,"cents")

la difference de price moyen entre le bus et le train pour les trajets entre 
 201-800 km est : 110.29435912813142 cents


In [500]:
mean_duration_800 = pd.Timedelta(seconds = df_800['duration']['mean']['bus']-df_800['duration']['mean']['train'])

print ("la difference de durée moyen entre le bus et le train pour les trajets entre \n 201-800 km est :",mean_duration_800)

la difference de durée moyen entre le bus et le train pour les trajets entre 
 201-800 km est : 0 days 00:53:34.985652


In [501]:
# difference de prix moyen selon le deplacement bus , train , sur 800-2000km

df_2000 = df.query('distance>=800' and 'distance<=2000').groupby('transport_type').agg({'duration': ['mean'],'price_in_cents' :['mean']})

In [506]:
df_2000

,duration,price_in_cents
,mean,mean
transport_type,,
bus,51290.188186,3333.038192
train,45733.333333,3265.555556


In [504]:
mean_price_2000 =  df_2000['price_in_cents']['mean']['bus'] - df_2000['price_in_cents']['mean']['train']
print ("la difference de price moyen entre le bus et le train pour les trajets entre \n 201-800 km est :",mean_price_2000,"cents")

la difference de price moyen entre le bus et le train pour les trajets entre 
 201-800 km est : 67.48263637741138 cents


In [507]:
mean_duration_2000 = pd.Timedelta(seconds = df_2000['duration']['mean']['bus']-df_2000['duration']['mean']['train'])

print ("la difference de durée moyen entre le bus et le train pour les trajets entre \n 201-800 km est :",mean_duration_)

la difference de durée moyen entre le bus et le train pour les trajets entre 
 201-800 km est : 0 days 01:32:36.854852


In [512]:
# difference de prix moyen selon le deplacement bus , train , sur 800-2000km

df_plus_2000 = df.query('distance >= 2000').groupby('transport_type').agg({'duration': ['mean'],'price_in_cents' :['mean']})

In [513]:
df_plus_2000 # pas de bus ou de train pour les trajets donc la distance est superieur a 2000 km

,duration,price_in_cents
,mean,mean
transport_type,,


<h5> <u>Conclusion</u></h5>

<p> pour cette deuxième partie nous avons fait de nouveau traitement tel que , le calcule de distance , faire un merge entre deux table. faire des calcules entre deux tables de façon simultané<br/> . puis on a répété des traitement de la  partie précédante.
</p>

<P> 
<font COLOR="red">-</font> notre grande supprises , nous remarquons que plus les distances des trajets sont courtes et plus long est le temps du trajet qu'on soit en bus ou en train et aussi la différence de prix entre un trajet en bus  et un trajet en train est plus élevé
<p/> 
    
<P> <font COLOR="red">-</font> les long trajets on une durée plus courte qu'on soit en bus ou en train mais on gagne quand même beaucoup plus de temps en train. Dans la moyen on gagne enviro 1h , c'est qui est préferable car il y'a pas une très grande différence de prix.
<p/>
   
<p>
    <u>
        remarque :
    </u>
    il y'a pas de bus sur les trajets de plus de 2000 km , ni de train non plus
</p>